## 飞桨PaddlePaddle X WeChaty AI ChatBot     创意赛

    
大赛链接: https://aistudio.baidu.com/aistudio/competition/detail/79    
    
    
PaddleHub是飞桨推出的预训练的深度学习模型的集合（Hub），在计算机视觉和文本方面都可以有非常多的实际应用。    
    
    
而Chatbot近几年来的商业应用也越来越多，WeChaty就是一个非常好用且支持多种语言多个平台的开源聊天机器人框架SDK，对开发Chatbot非常友好。   
    
    
      
### 赛题难点和重点
总的来说，就是一场应用场景的脑洞比赛，把Hub中的模型用到生活场景中。     
    
一开始会有很多想法，但事实上为了保证有一个比较好的模型质量必须先保证想要解决的问题的数据集的数量和质量。由于个人选手比较难找到高质量且专业的数据集，我们决定从现有的模型中选取一个相对成熟的深度学习网络，并尽量不在一开始就尝试fine tune。
    
第二个一开始比较困难的事情，是因为需要用到AI模型必须就要通过Python。而在python-Wechaty部署上我们遇到了不少麻烦。
    
![](https://ai-studio-static-online.cdn.bcebos.com/b1bf159be2a14b75b252ac1debc724b3cb90a6c709df4cfb8f173142aefb9990)
    
尝试了PadLocal的方式对接GateWay，但是由于是foreign IP，微信登陆的QRcode一直无法加载，最后只能选择使用docker + 免费Web协议。底层的对接实现是基于TypeScript语言，故无法直接在python-wechaty中使用该服务。可是Wechaty社区能够直接将其转化成对应的服务让多语言调用，从而实现：底层复用的特性。

整体步骤分为两步：

* 使用Docker启动web协议服务
* 使用python-wechaty连接服务

配置文件就在根目录下： [./wechaty_test.sh](wechaty_test.sh)
跟着[这个网页](https://python-wechaty.readthedocs.io/zh_CN/latest/introduction/use-web-protocol/)也可以轻松配置，只是稍微不同的是用来登陆微信的QR码不会出现在终端，而需要打开终端里出现的地址。
    
## 方案
出于模型质量和大小的考量，首先我们选择了PaddleHub的OCR模型(服务器端精度更高的版本)，想基于WeChaty实现一个图片中文字识别的微信机器人。     
换言之就是通过WeChaty将PaddleHub的OCR装到微信里 :)      
    
PaddleHub识别文字算法均采用CRNN（Convolutional Recurrent Neural Network）即卷积递归神经网络。其是DCNN和RNN的组合，专门用于识别图像中的序列式对象。与CTC loss配合使用，进行文字识别，可以直接从文本词级或行级的标注中学习，不需要详细的字符级的标注。该Module支持直接预测。 移动端与服务器端主要在于骨干网络的差异性，移动端采用MobileNetV3，服务器端采用ResNet50_vd。具体介绍可以参考PaddleHub[官方Notebook](https://aistudio.baidu.com/aistudio/projectdetail/507159)。    

CRNN的网络结构图：

<div align=center> <img src="https://ai-studio-static-online.cdn.bcebos.com/af68e45eea184b4c966f23ad7d9fd295e07e1fc31cc74134b4bd99ee275bed63"/>
    
    

OCR最基础的用法就是用户发送图片给chatbot，接受到图片之后会发送一条识别出来的所有文字消息回复。      
简单的例子就是快递单单号，身份证件号码，行程单号码日期，小红书的文字图片，笔记，等等的图片转换为文字（或者不知道怎么念但是需要打的字。。。）
     

实现基础功能之后，还可以再加上某一个行业的特定用途。通过python函数，或者另一个AI模型继续对识别出来的文字进行加工处理。
    
    
## 具体实现
### 第一步 安装包
这里提供aistudio可以运行的版本，在本机上运行的话在终端去掉前面的!就可以了。

In [1]:
#需要将PaddleHub和PaddlePaddle统一升级到2.0版本
!pip install paddlehub==2.0.0 -i https://pypi.tuna.tsinghua.edu.cn/simple 
!pip install paddlepaddle==2.0.0 -i https://pypi.tuna.tsinghua.edu.cn/simple 
#该Module依赖于第三方库shapely、pyclipper，使用该Module之前，请先安装shapely、pyclipper
!pip install shapely -i https://pypi.tuna.tsinghua.edu.cn/simple 
!pip install pyclipper -i https://pypi.tuna.tsinghua.edu.cn/simple 

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     |████████████████████████████████| 191 kB 12.9 MB/s eta 0:00:01
  Attempting uninstall: paddlehub
    Found existing installation: paddlehub 2.0.4
    Uninstalling paddlehub-2.0.4:
      Successfully uninstalled paddlehub-2.0.4
You should consider upgrading via the '/opt/conda/envs/python35-paddle120-env/bin/python -m pip install --upgrade pip' command.
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     |████████████████████████████████| 204.9 MB 30 kB/s  eta 0:00:0101| 107.1 MB 463 kB/s eta 0:03:31MB 433 kB/s eta 0:02:15 | 169.4 MB 461 kB/s eta 0:01:17 | 172.7 MB 464 kB/s eta 0:01:10��████████▋    | 176.7 MB 443 kB/s eta 0:01:04MB 125.7 MB/s eta 0:00:01
You should consider upgrading via the '/opt/conda/envs/python35-paddle120-env/bin/python -m pip install --upgrade pip' command.
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     |████████████████████████████████| 1.0 MB 19.8 MB/s eta 0:00:0

### 第二步 调用paddlehub 加载OCR预训练模型
这两行代码在test.py中可以看到，**paddlehub将预训练好的模型封装的好处就是可以直接放在实际应用的python代码中**。    
这里用移动端模型做例子，实际使用了服务端的模型（在test.py中可见）。

In [2]:
import paddlehub as hub

# 加载移动端预训练模型
ocr = hub.Module(name="chinese_ocr_db_crnn_mobile")
# 服务端可以加载大模型，效果更好 
# ocr = hub.Module(name="chinese_ocr_db_crnn_server") 

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:26: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def convert_to_list(value, n, name, dtype=np.int):
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/__init__.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import MutableMapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matp

Download https://bj.bcebos.com/paddlehub/paddlehub_dev/chinese_ocr_db_crnn_mobile_1.1.1.tar.gz
[##################################################] 100.00%
Decompress /home/aistudio/.paddlehub/tmp/tmpj6ukpols/chinese_ocr_db_crnn_mobile_1.1.1.tar.gz
[##################################################] 100.00%


[2021-04-29 08:08:45,543] [    INFO] - Successfully installed chinese_ocr_db_crnn_mobile-1.1.1
[2021-04-29 08:08:45,559] [ WARNING] - The _initialize method in HubModule will soon be deprecated, you can use the __init__() to handle the initialization of the object


### 第三步 识别图片

In [3]:
import cv2

# 读取测试文件夹test.txt中的照片路径
np_images =[cv2.imread("/home/aistudio/express.jpg")] 

results = ocr.recognize_text(
                    images=np_images,         # 图片数据，ndarray.shape 为 [H, W, C]，BGR格式；
                    use_gpu=False,            # 是否使用 GPU；若使用GPU，请先设置CUDA_VISIBLE_DEVICES环境变量
                    output_dir='ocr_result',  # 图片的保存路径，默认设为 ocr_result；
                    visualization=True,       # 是否将识别结果保存为图片文件；
                    box_thresh=0.5,           # 检测文本框置信度的阈值；
                    text_thresh=0.5)          # 识别中文文本置信度的阈值；

data = results[0]['data']
save_path = results[0]['save_path']
s = ""
for information in data:
    s += information['text']
    s += '\n'

print(s)

Download https://bj.bcebos.com/paddlehub/paddlehub_dev/chinese_text_detection_db_mobile_1.0.4.tar.gz
[##################################################] 100.00%
Decompress /home/aistudio/.paddlehub/tmp/tmpqo8n75k2/chinese_text_detection_db_mobile_1.0.4.tar.gz
[##################################################] 100.00%


[2021-04-29 08:08:53,552] [    INFO] - Successfully installed chinese_text_detection_db_mobile-1.0.4
[2021-04-29 08:08:53,558] [ WARNING] - The _initialize method in HubModule will soon be deprecated, you can use the __init__() to handle the initialization of the object
/home/aistudio/.paddlehub/modules/chinese_text_detection_db_mobile/processor.py:207: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  xmin = np.clip(np.floor(box[:, 0].min()).astype(np.int), 0, w - 1)
/home/aistudio/.paddlehub/modules/chinese_text_detection_db_mobile/processor.py:208

EXPRESS
S
顺丰速运
第1次打印打印时间01月03日23:43
运单号：SF101569720
010WB-01
218/250
8个人
张翔宇157****8458
北京市北京市
西里小区6号楼4单元
寄付月结
I1B
任艳蕊税务********9529自如
北京市朝阳区酒仙桥将台路5号院******
增值服务：
件数：1
月结卡号：01******92
实际重量：1kg
运费：13（不作结算依据）
费用合计：
托寄物：发票数量：1
备注：25254657



### 第四步 效果展示

在完成第三部分的一键OCR预测之后，由于我们设置了visualization=True，所以我们会自动将识别结果保存为图片文件，并默认保存在ocr_result文件夹中。**刷新即可获取到新生成的ocr_result文件夹。**

![](https://ai-studio-static-online.cdn.bcebos.com/951e88e87f66423d8fa6ccc1e9f10cc2036a91a59cd5448bbff2a99d32a4045c)

识别前与识别后的结果：


In [4]:
import matplotlib.pyplot as plt 
import matplotlib.image as mpimg 

# 识别前的图片 与 识别后的效果图
test_img_path = ["./express.jpg", "./ocr_result/ndarray_1619654937.8608792.jpg"]


img1 = mpimg.imread(test_img_path[0]) 
plt.figure(figsize=(10,10))
plt.imshow(img1) 
plt.axis('off') 
plt.show()


img1 = mpimg.imread(test_img_path[1]) 
plt.figure(figsize=(10,10))
plt.imshow(img1) 
plt.axis('off') 
plt.show()

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/cbook/__init__.py:2349: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  if isinstance(obj, collections.Iterator):
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/cbook/__init__.py:2366: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  return list(data) if isinstance(data, collections.MappingView) else data


<Figure size 1000x1000 with 1 Axes>

<Figure size 1000x1000 with 1 Axes>

接下来的改进方向则是根据某个行业方向加上有意思或者别的量化指标，比如根据食品的营养成分表进行饮食推荐。

### References
* python-WeChaty使用相关参考github repo（有很多python实例在/examples子文件夹中）: [https://github.com/wechaty/python-wechaty](https://github.com/wechaty/python-wechaty)    

* PaddleHub一键OCR中文识别（超轻量8.1M模型，火爆）: [https://aistudio.baidu.com/aistudio/projectdetail/507159](https://aistudio.baidu.com/aistudio/projectdetail/507159)    

* 另外对OCR感兴趣的同学可以参加目前的这个比赛：[https://aistudio.baidu.com/aistudio/competition/detail/75](https://aistudio.baidu.com/aistudio/competition/detail/75)    

    
### Roadmap
第一稿（2021/04/29）：我们先从实现基础架构开始 :)


请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 